## Line identification

The primary function to make line identifications with input line list is the class `Line_list` from `pyemili.Lines`. So import this class when you need to identify lines.

In [1]:
from pyemili.Lines import Line_list
import pandas as pd

Here we put the input line list file at the same directory and load it.

In [2]:
hf22 = pd.read_table('Hf2-2_linelist.txt',delim_whitespace=True)


In [3]:
hf22

,wav_cor,dered_flux,snr,fwhm
0,3132.839,925.120,16.960,0.308
1,3187.780,1793.079,24.820,0.468
2,3203.148,594.023,13.200,0.354
3,3218.232,330.976,7.338,0.488
4,3244.051,204.894,3.571,0.861
...,...,...,...,...
413,9990.132,37.357,10.360,1.883
414,9991.311,22.606,9.655,1.386
415,10027.628,19.509,5.824,1.230
416,10049.035,290.967,37.780,1.502


The first column is the corrected wavelength, the second column is the deredded flux, and the third column is the SNR, and the fourth column is the FWHM in angstroms. PyEMILI needs the wavelength and flux primarily. It is also okay the input line list does not contain SNR and FWHM columns.
Here we get the wavelength, flux, SNR, and FWHM:

In [4]:
wavelength = hf22.wav_cor
flux = hf22.dered_flux
snr = hf22.snr
fwhm = hf22.fwhm


Now we generate an instance with the above information

In [5]:
hf22_out = Line_list(wavelength=wavelength,wavelength_error=10,flux=flux,snr=snr,fwhm=fwhm)

In this instance, we can use the function `identify` to automatically identify the input lines.

In [6]:
hf22_out.identify('Hf2-2',abun_type='nebula')

Initializing database
Use Nebula Abundance
Using Default ICF
Using Default Velocity Matrix
Initializing Done
Number of Iterations: 1
ICF Values: Bin/%          Velocity Structure: Bin/Vel(km/s)
ix 1:  0.9900              irvcor 1:  0.000
ix 2:  49.504              irvcor 2:  0.000
ix 3:  39.603              irvcor 3:  0.000
ix 4:  9.9008              irvcor 4:  0.000
ix 5:  0.0009              irvcor 5:  0.000



SubProcessing:: 100%|██████████| 418/418 [00:10<00:00, 40.99it/s]


H beta flux is not normalized. About to normalize the flux.
Number of Iterations: 2
ICF Values: Bin/%          Velocity Structure: Bin/Vel(km/s)
ix 1:  1.0                 irvcor 1:  -0.251
ix 2:  33.749              irvcor 2:  0.134
ix 3:  63.753              irvcor 3:  0.243
ix 4:  1.4940              irvcor 4:  0.439
ix 5:  0.0025              irvcor 5:  5.317



MainProcessing:: 100%|██████████| 418/418 [00:10<00:00, 39.57it/s]


You can see in the terminal that the code starts its first line identifications with default ICF and Velocity Structure parameters, and then starts the second line identifications with optimized parameters derived from the first results. This is what the code will do when setting `iteration` to True. After the second iteration has done, two files end with '.dat' and '.out' will be generated in the same directory. In this example it will be 'Hf2-2.dat' and 'Hf2-2.out'. More description about the output files can be found [here](./Intro.md).

Another example:

In [ ]:
J0608 = pd.read_table('J0608_linelist.txt',delim_whitespace=True)
J0608_out = Line_list(wavelength=J0608.wave_cor.values,wavelength_error=30,flux=J0608.F.values,snr=J0608.snr.values,fwhm=J0608.fwhm.values)
J0608_out.identify('J0608',icf=[0.01,0.5,0.4,0.1,0.00001],Te=30000,abun_type='abun_WC.dat',deplete='O 50 3')

In this condition, the code will use fixed `icf` values specified by the user. The electron temperature will be set to 30000K to calculate the predicted template flux of each transition. The elemental abundance table will be used from the 'abun_WC.dat' file. And the abundances of ion O III will be reduced by a factor of 50.

## Parameters of `pyemili.Lines.Line_list.identify()`


>**filename** : str, file-like or `pathlib.Path`
>>Name of generated files.

>**icf** : array_like, optional
>>The percentage of each energy bin. The array length of this parameter must be 5 (the same as energy bin). These values are percentages of particular ions of that elements take up of the entire abundance for that element. Default values are `[0.01,0.5,0.4,0.1,0.00001]`. NOTE: If `iteration` is `True`, the `icf` will be calculated automatically based on the first iteration. If both `iteration` is `True` and `icf` is specified, code will prefer the specified `icf`.

>**v_cor** : array_like, optional
>>The velocity correction of each energy bin. The array length of this parameter must be 5 (the same as energy bin). Default values are `[0,0,0,0,0]`. NOTE: If `iteration` is `True`, `v_cor` will be calculated automatically. If both `iteration` is `True` and `v_cor` is specified, code will prefer the specified `v_cor`.

>**sigma** : int, optional
>>The maximum times of the wavelength uncertainty used for line identifications. e.g., `sigma` = 5 means candidates within 5 times wavelength uncertainty of observed line will be included.

>**Ne** : int, optional
>>The electron density in the unit of cm^-3. Default value is 10000.

>**Te** : int, optional
>>The electron temperature in the unit of K. Default value is 10000.

>**I** : int, optional
>>The instrumental resolution of the spectrum in the unit of km/s. Multiplet lines whose wavelengths are within `I` will be treated as one multiplet line in the "Multiplet Check" part.

>**deplete** : str, optional 
>>To reduce or enhance the abundance of certain ions after the ionic abundances have been calculated using the `icf` values. This parameter has up to 4 arguments specified in the following order:
>>    1. The element to deplete or enhance.
>>    2. The amount to deplete or enhance (negative value enhance).
>>    3. The lower end of the range of ionization states to deplete or enhance.
>>    4. The upper end of the range of ionization states to deplete or enhance.
>>Different arguments should be separated with space. Different ions should be separated with comma. 
>>Examples:
>>* `Fe 10 1 3` This input type will reduce the abundances of Fe I, Fe II, and Fe III by a factor of 10.
>>* `Fe -10 1` This input type will enhance the abundance of Fe I by a factor of 10.
>>* `Fe 20, Ti 10` This input type will reduce the abundances of all ion of Fe and Ti by a factor of 20 and 10 respectively.

>**abun_type** : str, file-like or `pathlib.Path`, optional
>>The input elemental abundance table. Here are 2 options: 'solar' [(M. Asplund et al. 2009)](https://ui.adsabs.harvard.edu/abs/2009ARA%26A..47..481A/abstract), 'nebula' [(Osterbrock & Ferland 2006)](https://ui.adsabs.harvard.edu/abs/2006agna.book.....O/abstract). Default is 'solar'. Besides, you can use the filename of other abundance file as input.

>**iteration** : bool
>>If True, code will extract a sub line list with relatively robust identifications made in the first iteration to calculate the new `icf` and `v_cor` models and re-identify each line. Default is True.

>**match_list** : str, optional 
>>The filename of the input match line table. This is a obsolete argument. It is in conflict with `iteration` parameter and may be deleted in future version. See [Sharpee et al. (2003)](https://ui.adsabs.harvard.edu/abs/2003ApJS..149..157S/abstract) for more details.